# 機械学習を用いた予測

## ライブラリのインポート

In [5]:
# データセット分割
from sklearn.model_selection import train_test_split
# 標準化
from sklearn.preprocessing import StandardScaler
# 正答率
from sklearn.metrics import accuracy_score

# K-近傍法
from sklearn.neighbors import KNeighborsClassifier
# サポートベクターマシン
from sklearn.svm import SVC
# 決定木
from sklearn.tree import DecisionTreeClassifier
# ランダムフォレスト、Adaブースト
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# ナイーブベイズ
from sklearn.naive_bayes import GaussianNB
# 線形判別分析
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# 二次判別分析
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from mlxtend.plotting import plot_decision_regions

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import talib as ta

from df_method import rise_fall_rate, moving_average, GCDC, df_shift, add_data, RSI, Z_score_normalization, Min_Max_normalization

## 分類器と対応する名前の用意

In [6]:
names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Decision Tree",
    "Random Forest",
    "AdaBoost",
    "Naive Bays",
    "Linear Discriminant Analysis",
    "Quadratic Discriminant Analysis"
]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5),
    AdaBoostClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis()
]

## 変数宣言

In [7]:
csv_path = './csv_realtime/'
x_days_later = 1
start_year = '2013'
end_year = '2018-10-31'

## データの用意

In [26]:
result_csv = pd.DataFrame(columns=['day', 'model', 'year', 'maxlen', 'n_hidden', 'time', 'correct'])

df_list = []

#df_list.append(add_data(csv_path+'NASDAQ.csv', x_days_later))
#df_list.append(add_data(csv_path+'USD_JPY.csv', x_days_later)) # ドル円
#df_list.append(add_data(csv_path+'EUR_JPY.csv', x_days_later))
#df_list.append(add_data(csv_path+'EUR_USD.csv', x_days_later))
#df_list.append(add_data(csv_path+'nikkei.csv', x_days_later))
df_list.append(add_data(csv_path+'DOW30.csv', x_days_later))

# ドル円の終値と始値の差
df = pd.read_csv(csv_path+'USD_JPY.csv', index_col='Date', parse_dates=True)
df = df.apply(np.log)*100
df = df['USD_JPY_Close'] - df['USD_JPY_Open']
df = df['1989-10-16':]
df = df.rename('USD_JPY_diff')
df_list.append(df)

#ドル円の移動平均を追加する実験
#df = pd.read_csv(csv_path+'USD_JPY.csv', index_col='Date', parse_dates=True)
#df_list.append(GCDC(df['USD_JPY_Close'], 5, 25))
#df_list.append(GCDC(df['USD_JPY_Close'], 25, 75))

#ドル円のRSI
#df = pd.read_csv(csv_path+'USD_JPY.csv', index_col='Date', parse_dates=True)
#df = RSI(df['USD_JPY_Close'])
##df = df.apply(np.log) # 対数をとる
#df_list.append(df)
##df_list.append(rise_fall_rate(df, x_days_later))

# すべて一つのデータフレームに統合する
df_x = df_list[0]
for i in range(len(df_list) - 1):
    df = df_list[i + 1]
    df_x = df_x.join(df, how='inner', rsuffix='_' + str(i))

#df_x = df_x[['_Close', 'USD_JPY_diff']]# ●デバッグ
print(df_x)

            DOW_Open  DOW_High   DOW_Low  DOW_Close  DOW_Adj Close  \
Date                                                                 
1989-10-16 -8.703912 -3.894794 -1.926121   3.372271       3.372271   
1989-10-17  4.461968 -0.076876  3.610545  -0.704290      -0.704290   
1989-10-18 -0.459187 -0.038652  0.843563   0.186277       0.186277   
1989-10-19  1.217601  1.609887  1.518681   1.484959       1.484959   
1989-10-20  0.943253 -0.166709  0.363026   0.221131       0.221131   
1989-10-23 -0.084232  0.060649 -0.432471  -0.980192      -0.980192   
1989-10-24 -1.090574 -0.890181 -3.008054  -0.138664      -0.138664   
1989-10-25 -0.255144  0.152830  2.207883  -0.223621      -0.223621   
1989-10-26 -0.131978 -1.005097 -1.145185  -1.501831      -1.501831   
1989-10-27 -1.507935 -1.397226 -0.935166  -0.652921      -0.652921   
1989-10-30  0.000000  0.226363  0.563774   0.259990       0.259990   
1989-10-31  0.455724  1.325000  0.726070   1.585234       1.585234   
1989-11-01  0.974604

In [27]:
if end_date == '':
    df_x = df_x[start_date:]
else:
    df_x = df_x[start_date:end_date]

target = df_x['USD_JPY_diff']
print(target)

df_t = pd.DataFrame(index=df_x.index, columns=['t'])
df_t = df_t.fillna(0)

df_t.loc[0 <= target, 't'] = 1
df_t.loc[target < 0, 't'] = 0

df_t = df_t.shift(-1 * x_days_later, axis=0)
df_t = df_t.drop(df_t.index[-1*x_days_later:], axis=0)
df_x = df_x.drop(df_x.index[-1*x_days_later:], axis=0)

df_t = df_t['t']
df_t = df_t.astype('int64')
print(df_t)

Date
2010-01-04   -0.183536
2010-01-05   -0.977313
2010-01-06    0.717628
2010-01-07    1.066878
2010-01-08   -0.753178
2010-01-11   -0.530620
2010-01-12   -1.234516
2010-01-13    0.416850
2010-01-14   -0.208208
2010-01-15   -0.351649
2010-01-19    0.395909
2010-01-20    0.098733
2010-01-21   -0.870002
2010-01-22   -0.587943
2010-01-25    0.388652
2010-01-26   -0.678346
2010-01-27    0.378662
2010-01-28   -0.077843
2010-01-29    0.410769
2010-02-01    0.409089
2010-02-02   -0.276228
2010-02-03    0.628690
2010-02-04   -2.156118
2010-02-05    0.358865
2010-02-08   -0.078427
2010-02-09    0.480797
2010-02-10    0.311839
2010-02-11   -0.211499
2010-02-12    0.255968
2010-02-16    0.111025
                ...   
2018-10-01    0.237206
2018-10-02   -0.246024
2018-10-03    0.753861
2018-10-04   -0.472938
2018-10-05   -0.140622
2018-10-08   -0.414262
2018-10-09   -0.221112
2018-10-10   -0.612655
2018-10-11   -0.106952
2018-10-12    0.071339
2018-10-15   -0.366186
2018-10-16    0.419662
2018-1

In [28]:
x_data = df_x.values
t_data = df_t.values
print(x_data.shape)
print(x_data)
print(t_data.shape)
print(t_data)

(2230, 7)
[[-1.12321322e+00  4.72212687e-01  7.25099499e-02 ...  1.48403717e+00
   2.64915073e+01 -1.83535814e-01]
 [ 1.46438315e+00 -1.92643858e-01  8.76521441e-01 ... -1.12879966e-01
   4.75763048e+00 -9.77312589e-01]
 [-1.87617257e-01  9.84975124e-02  2.28109596e-01 ...  1.57020772e-02
  -1.33484816e+00  7.17628392e-01]
 ...
 [ 7.54660047e-01  5.64942865e-01  7.23500752e-01 ...  6.78365457e-01
  -2.31927612e+01  2.11827087e-01]
 [ 1.31002031e+00  2.11487189e+00  1.25358240e+00 ...  2.10482632e+00
   3.09609793e+01  1.23380645e-01]
 [ 1.35238759e+00  2.96047115e-01  1.21904328e+00 ...  4.17017564e-02
  -2.01859970e+01  4.48253882e-01]]
(2230,)
[0 1 1 ... 1 1 0]


## データの分割と標準化

In [29]:
"""
#x_data = StandardScaler().fit_transform(x_data)

# 学習データとラベルデータを訓練データとテストデータに分割する
x_train, x_test, t_train, t_test = train_test_split(x_data, t_data, test_size=0.1, random_state=None)

# 標準化
sc = StandardScaler()
sc.fit(x_train)
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)
"""

'\n#x_data = StandardScaler().fit_transform(x_data)\n\n# 学習データとラベルデータを訓練データとテストデータに分割する\nx_train, x_test, t_train, t_test = train_test_split(x_data, t_data, test_size=0.1, random_state=None)\n\n# 標準化\nsc = StandardScaler()\nsc.fit(x_train)\nx_train_std = sc.transform(x_train)\nx_test_std = sc.transform(x_test)\n'

## メイン処理

In [30]:
result_csv = pd.DataFrame(columns=names)

for i in range(100):
    print('試行回数：', i)
    
    # 学習データとラベルデータを訓練データとテストデータに分割する
    # x_train, x_test, t_train, t_test = train_test_split(x_data, t_data, test_size=0.1, random_state=None)
    n_train = int(len(df_x)*0.9)              # 訓練データ長
    #print(n_train)
    x_train,x_test = np.vsplit(np.array(x_data), [n_train])  # 学習データを訓練用とテスト用に分割
    t_train,t_test = np.split(np.array(t_data), [n_train])

    # 標準化
    sc = StandardScaler()
    sc.fit(x_train)
    x_train_std = sc.transform(x_train)
    x_test_std = sc.transform(x_test)
    
    result_csv.loc[i] = 0
    for name, model in zip(names, classifiers):
        model.fit(x_train_std, t_train) # 学習
        
        # 正答率を表示
        print(name)
        pred_train = model.predict(x_train_std)
        accuracy_train = accuracy_score(t_train, pred_train)
        print('訓練データでの正答率: %.2f' % accuracy_train)
        pred_test = model.predict(x_test_std)
        accuracy_test = accuracy_score(t_test, pred_test)
        print('テストデータでの正答率: %.2f' % accuracy_test)
        print("\n")
    
        # 入力次元が２の場合のみグラフを表示
        if (len(df_x.columns) == 2):
            plt.style.use('ggplot')
    
            x_combined_std = np.vstack((x_train_std, x_test_std))
            y_combined = np.hstack((t_train, t_test))

            fig = plt.figure(figsize=(13, 8))
            plot_decision_regions(x_combined_std, y_combined, clf=model, res=0.02)
            plt.show()
    
        # ログを書き込む
        result_csv.at[i, name] = accuracy_test
        result_csv.to_csv('./log_scikit_learn/log.csv', index=False)

試行回数： 0
Nearest Neighbors
訓練データでの正答率: 0.76
テストデータでの正答率: 0.50


Linear SVM
訓練データでの正答率: 0.52
テストデータでの正答率: 0.43


RBF SVM
訓練データでの正答率: 0.86
テストデータでの正答率: 0.52


Decision Tree
訓練データでの正答率: 0.58
テストデータでの正答率: 0.49


Random Forest
訓練データでの正答率: 0.64
テストデータでの正答率: 0.54


AdaBoost
訓練データでの正答率: 0.60
テストデータでの正答率: 0.48


Naive Bays
訓練データでの正答率: 0.52
テストデータでの正答率: 0.46


Linear Discriminant Analysis
訓練データでの正答率: 0.53
テストデータでの正答率: 0.44


Quadratic Discriminant Analysis
訓練データでの正答率: 0.49
テストデータでの正答率: 0.52


試行回数： 1
Nearest Neighbors
訓練データでの正答率: 0.76
テストデータでの正答率: 0.50


Linear SVM
訓練データでの正答率: 0.52


C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


テストデータでの正答率: 0.43


RBF SVM
訓練データでの正答率: 0.86
テストデータでの正答率: 0.52


Decision Tree
訓練データでの正答率: 0.58
テストデータでの正答率: 0.49


Random Forest
訓練データでの正答率: 0.67
テストデータでの正答率: 0.49


AdaBoost
訓練データでの正答率: 0.60
テストデータでの正答率: 0.48


Naive Bays
訓練データでの正答率: 0.52
テストデータでの正答率: 0.46




C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Linear Discriminant Analysis
訓練データでの正答率: 0.53
テストデータでの正答率: 0.44


Quadratic Discriminant Analysis
訓練データでの正答率: 0.49
テストデータでの正答率: 0.52


試行回数： 2
Nearest Neighbors
訓練データでの正答率: 0.76
テストデータでの正答率: 0.50


Linear SVM
訓練データでの正答率: 0.52
テストデータでの正答率: 0.43


RBF SVM
訓練データでの正答率: 0.86
テストデータでの正答率: 0.52


Decision Tree
訓練データでの正答率: 0.58
テストデータでの正答率: 0.49


Random Forest
訓練データでの正答率: 0.66
テストデータでの正答率: 0.53


AdaBoost
訓練データでの正答率: 0.60
テストデータでの正答率: 0.48


Naive Bays
訓練データでの正答率: 0.52
テストデータでの正答率: 0.46


Linear Discriminant Analysis
訓練データでの正答率: 0.53
テストデータでの正答率: 0.44


Quadratic Discriminant Analysis
訓練データでの正答率: 0.49
テストデータでの正答率: 0.52


試行回数： 3
Nearest Neighbors
訓練データでの正答率: 0.76
テストデータでの正答率: 0.50


Linear SVM


...

C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Linear SVM
訓練データでの正答率: 0.52
テストデータでの正答率: 0.43


RBF SVM
訓練データでの正答率: 0.86
テストデータでの正答率: 0.52


Decision Tree
訓練データでの正答率: 0.58
テストデータでの正答率: 0.49


Random Forest
訓練データでの正答率: 0.62
テストデータでの正答率: 0.53


AdaBoost
訓練データでの正答率: 0.60
テストデータでの正答率: 0.48


Naive Bays
訓練データでの正答率: 0.52
テストデータでの正答率: 0.46


Linear Discriminant Analysis
訓練データでの正答率: 0.53
テストデータでの正答率: 0.44


Quadratic Discriminant Analysis
訓練データでの正答率: 0.49
テストデータでの正答率: 0.52


試行回数： 99
Nearest Neighbors
訓練データでの正答率: 0.76
テストデータでの正答率: 0.50




C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Linear SVM
訓練データでの正答率: 0.52
テストデータでの正答率: 0.43


RBF SVM
訓練データでの正答率: 0.86
テストデータでの正答率: 0.52


Decision Tree
訓練データでの正答率: 0.58
テストデータでの正答率: 0.49


Random Forest
訓練データでの正答率: 0.65
テストデータでの正答率: 0.56


AdaBoost
訓練データでの正答率: 0.60
テストデータでの正答率: 0.48


Naive Bays
訓練データでの正答率: 0.52
テストデータでの正答率: 0.46


Linear Discriminant Analysis
訓練データでの正答率: 0.53
テストデータでの正答率: 0.44


Quadratic Discriminant Analysis
訓練データでの正答率: 0.49
テストデータでの正答率: 0.52




C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\15rd112\AppData\Local\Continuum\anaconda3\envs\deeplearning\lib\site-packages\sklearn\discriminant_analysis.py:692: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [31]:
for name in names:
    print(name)
    print('正答率')
    print(result_csv[name].mean())
    print("\n")

Nearest Neighbors
正答率
0.4977578475336326


Linear SVM
正答率
0.43497757847533586


RBF SVM
正答率
0.5156950672645736


Decision Tree
正答率
0.48878923766816257


Random Forest
正答率
0.5147982062780269


AdaBoost
正答率
0.47533632286995514


Naive Bays
正答率
0.46188340807174905


Linear Discriminant Analysis
正答率
0.44394618834080757


Quadratic Discriminant Analysis
正答率
0.5156950672645736


